In [12]:
#Optional module
#Checks the encoding of inputed data data
import chardet
with open(input('Enter dataset '), 'rb') as rawdata:
    result = chardet.detect(rawdata.read(1000000))
result

Enter datasethuge_train_tweets.csv


{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}

In [2]:
#Libraries
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

#Importing data
training_set = input('Filename of training set ')
train_label = input('Column containing labels of training data ')
train_tweets = input('Column containing tweets of training data ')
train = pd.read_csv(training_set)

#Prints columns of imported dataset
columns = train.columns.values
print(columns)
#train.head()

Filename of training sethuge_train_tweets.csv
Column containing labels of training datatarget
Column containing tweets of training datatext 
['target' 'id' 'date' 'flag' 'user' 'text ']


In [22]:
positive=[]
negative=[]
dontknow=[]
i=0
while i<len(train):
    if train['target'][i] == 0:
        positive.append(train['target'][i])
        i+=1
    elif train['target'][i] == 1:
        negative.append(train['target'][i])
        i+=1
    else: 
        dontknow.append('dont work')
        i+=1
        
print(len(positive))
print(len(negative))

800000
800000


In [3]:
#Optional module
#Splits multinominal data into binominal data
mean = train[train_label].mean()
binary_target = []

for element in train[train_label]:
    if element >= mean:
        binary_target.append(1)
    elif element < mean:
        binary_target.append(0)
train[train_label] = binary_target
train.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
def preprocessing(tweet):     
    #Tokenization
    def form_sentence(tweet):
        tweet_blob = TextBlob(tweet)
        return ' '.join(tweet_blob.words)
    formed_tweet = form_sentence(tweet)
    #Removes words and symbols with little to no 
    #sentimental value
    def clean_tweets(tweet):
        tweet_split = [wrd for wrd in tweet.split()]
        clean_wrds = [s for s in tweet_split if re.match(r'[^\W\d]*$', s)]
        clean_s = ' '.join(clean_wrds)
        clean = [word for word in clean_s.split() if word.lower() not in stopwords.words('english')]
        return clean
    clean_tweet = clean_tweets(formed_tweet)
    #Normalizes the tweets
    #eg.replaces words with same meaning
    def normalize(tweets):
        lem = WordNetLemmatizer()
        normalized_tweet = []
        for word in tweets:
            normalized_text = lem.lemmatize(word,'v')
            normalized_tweet.append(normalized_text)
        return normalized_tweet
    return normalize(clean_tweet)

In [5]:
#Pipeline for data to go through
pipeline = Pipeline(steps=[
    ('bow', CountVectorizer(analyzer = preprocessing)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultinomialNB()),
])

In [6]:
#Splits data into test and training datasets
tweet_train, tweet_test, label_train, label_test = train_test_split(train[train_tweets], train[train_label], test_size=0.2)
pipeline.fit(tweet_train,label_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function preprocessing at 0x7fb8c2d981f0>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [7]:
#Predictions of train / test split
predictions = pipeline.predict(tweet_test)
#print(classification_report(predictions,label_test))
print(accuracy_score(predictions, label_test))
print(predictions)

0.75923125
[0 1 1 ... 1 1 0]


In [8]:
#Predictions of another dataset
#Pretty horible results
new_tweets = pd.read_csv('train_tweets.csv')
new_predictions = pipeline.predict(new_tweets['tweet'])
print(accuracy_score(new_predictions, new_tweets['label']))

0.40554408359927413
